In [1]:
import basicLib.loadAndTest as orig
import basicLib.featureUpdate as featUp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import *
from pylab import *
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import re
from sklearn.linear_model import LogisticRegression
import basicLib.modelRun as runms
from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time
import clf2net
import clf2netTest
import clf2netTree
from sklearn import ensemble

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pathTrain = '../input_clean/users_train_final.csv'
pathTest = '../input_clean/users_test_final.csv'
pathyRes = '../input_clean/yRes_final.csv'

In [3]:
finalTrain = pd.read_csv(pathTrain)
finalTest = pd.read_csv(pathTest)
yRes = pd.read_csv(pathyRes)

In [4]:
finalTrainUse = finalTrain.drop('id',axis=1)
finalTestUse = finalTest.drop('id',axis=1)

### training

In [5]:
predictMethod = MLPClassifier(solver='lbfgs', alpha=1e-5)
predictMethod1 = LogisticRegression()
predictMethod2 = tree.DecisionTreeClassifier()

In [6]:
#clf2netTree.clf2netTree((predictMethod1,predictMethod2),predictMethod , finalTrainUse,yRes['country_destination'])
clfList = clf2netTest.runClf2net((predictMethod1,predictMethod2),predictMethod, finalTrainUse,yRes['country_destination'] )

-----------The clf is: splitTrain- False , useXtrain- False -----------
The accuracy score of the clf is:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) ##### The score is: 0.698493551534
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best') ##### The score is: 0.576948087136
MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momen

In [ ]:
y_pred = predictMethod.predict(X_test)
accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = predictMethod.predict_proba(X_test)

In [ ]:
accuracy_score

In [ ]:
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    myArr.append(predictMethod.classes_[order[:5]])

In [14]:
len(finalTrainUse)

213451

### clf2net

In [5]:
predictMethod = MLPClassifier(solver='lbfgs', alpha=1e-5)
predictMethod1 = LogisticRegression()
predictMethod2 = tree.DecisionTreeClassifier()

##### false, false

In [6]:
start = time.time()
myClf = clf2net.clf2net((predictMethod1,predictMethod2),predictMethod)
myClf.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

426.977187871933


In [39]:
start = time.time()
y_pred = myClf.predict(finalTestUse)
end = time.time()
print(end-start)
#y_pred = myClf.get_y_pred()
#accuracy_score = metrics.accuracy_score(y_test, y_pred)
y_pred_proba_temp = myClf.predict_proba(finalTestUse)

0.673255443572998


In [32]:
idCol = finalTest['id']
idCol = idCol.reset_index(drop=True)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_clf2netFF_S.csv',index=False)

##### false, true

In [33]:
start = time.time()
myClf_FT = clf2net.clf2net((predictMethod1,predictMethod2),predictMethod,useXtrain = True)
myClf_FT.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

574.121209859848


In [36]:
start = time.time()
y_pred = myClf_FT.predict(finalTestUse)
end = time.time()
print(end-start)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_clf2netFT_S.csv',index=False)

1.5759568214416504


##### true, false

In [35]:
start = time.time()
myClf_TF = clf2net.clf2net((predictMethod1,predictMethod2),predictMethod,splitTrain = True)
myClf_TF.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

201.41696667671204


In [37]:
start = time.time()
y_pred = myClf_TF.predict(finalTestUse)
end = time.time()
print(end-start)
d = {'id' : idCol, 'country' : pd.Series(y_pred)}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_clf2netTF_S.csv',index=False)

0.5930314064025879


##### true, true

In [6]:
start = time.time()
myClf_TT = clf2net.clf2net((predictMethod1,predictMethod2),predictMethod,splitTrain = True, useXtrain = True)
myClf_TT.fit(finalTrainUse,yRes['country_destination'])
end = time.time()
print(end-start)

243.1120901107788


In [40]:
start = time.time()
myClf_TT.predict(finalTestUse)
end = time.time()
print(end-start)
y_pred = myClf_TT.get_y_pred()
y_pred_proba_temp = myClf_TT.predict_proba(finalTestUse)
d = {'id' : idCol, 'country' : pd.Series(y_pred[0])}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_clf2netTT_regression_S.csv',index=False)
d = {'id' : idCol, 'country' : pd.Series(y_pred[1])}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_clf2netTT_tree_S.csv',index=False)
d = {'id' : idCol, 'country' : pd.Series(y_pred[2])}
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_clf2netTT_S.csv',index=False)

1.2680974006652832


#### to put 5 destination

In [8]:
start = time.time()
y_pred_proba_temp = myClf_TT.predict_proba(finalTestUse)
end = time.time()
print(end-start)
myArr = []
for x in y_pred_proba_temp:
    order = np.argsort(x)[::-1]
    for i in myClf_TT.classes_()[order[:5]]:
        myArr.append(i)
idCol = finalTest.id.repeat(5)
idCol = idCol.reset_index(drop=True)

1.2049965858459473


In [9]:
d = {'id' : idCol , 'country' : pd.Series(myArr)}

In [10]:
df = pd.DataFrame(d,columns=['id', 'country'])
df.to_csv('../submission/submission_clf2netTT_M.csv',index=False)